In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("catalogo", "catalog_jptq")
dbutils.widgets.text("esquema_source", "bronze_jptq")
dbutils.widgets.text("esquema_stage", "silver_jptq")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_stage = dbutils.widgets.get("esquema_stage")

In [0]:
def fn_transform_director_pelicula(catalogo: str, esquema_source: str):
   sql_select_director = f"""SELECT
                              CONCAT(
                              SUBSTRING(SPLIT(DES_DIRECTOR,' ')[0],1,3),
                              SUBSTRING(ELEMENT_AT(SPLIT(DES_DIRECTOR,' '),-1),1,3)
                              ) AS COD_DIRECTOR,
                              DES_DIRECTOR
                              FROM {catalogo}.{esquema_source}.TBL_DETALLE_PELICULAS
                              GROUP BY DES_DIRECTOR
                              ORDER BY DES_DIRECTOR ASC"""
   df_t_director_pelicula = spark.sql(sql_select_director)
   return df_t_director_pelicula

In [0]:
df_tmp_director_pelicula = fn_transform_director_pelicula(catalogo, esquema_source)

In [0]:
df_tmp_director_pelicula = df_tmp_director_pelicula.withColumn("FEC_CARGA", current_timestamp())

In [0]:
display(df_tmp_director_pelicula)

In [0]:
df_tmp_director_pelicula.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_stage}.TBL_TMP_DIRECTOR_PELICULA")

In [0]:
%sql
SELECT
*
FROM
catalog_jptq.silver_jptq.tbl_tmp_director_pelicula